In [1]:
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
train_dir = '/kaggle/input/vehicle-detection-image-set/data'

In [3]:
dataGenerator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.2)

training_data = dataGenerator.flow_from_directory(train_dir, batch_size=64, target_size=(224, 224), shuffle=True, subset='training')
validation_set = dataGenerator.flow_from_directory(train_dir, batch_size=64, target_size=(224, 224), shuffle=True, subset='validation')

Found 14209 images belonging to 2 classes.
Found 3551 images belonging to 2 classes.


In [4]:
mobile = tf.keras.applications.mobilenet.MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

17225924/17225924 [==============================] - 1s 0us/step


In [5]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [6]:
for layer in mobile.layers:
    layer.trainable = False

In [7]:
base_output = mobile.output

gap = tf.keras.layers.GlobalAveragePooling2D()(base_output)
fc1 = tf.keras.layers.Dense(128, activation='relu')(gap)
output = tf.keras.layers.Dense(2, activation='softmax')(fc1)

model = tf.keras.models.Model(inputs = mobile.input, outputs = output)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(training_data, epochs=5, verbose=1, validation_data=validation_set)

Epoch 1/5
223/223 [==============================] - 96s 398ms/step - loss: 0.0273 - accuracy: 0.9909 - val_loss: 0.0080 - val_accuracy: 0.9980
Epoch 2/5
223/223 [==============================] - 49s 219ms/step - loss: 0.0033 - accuracy: 0.9992 - val_loss: 0.0037 - val_accuracy: 0.9992
Epoch 3/5
223/223 [==============================] - 48s 217ms/step - loss: 0.0014 - accuracy: 0.9996 - val_loss: 0.0038 - val_accuracy: 0.9992
Epoch 4/5
223/223 [==============================] - 47s 211ms/step - loss: 6.8623e-04 - accuracy: 0.9999 - val_loss: 0.0039 - val_accuracy: 0.9992
Epoch 5/5
223/223 [==============================] - 46s 208ms/step - loss: 3.5278e-04 - accuracy: 1.0000 - val_loss: 0.0030 - val_accuracy: 0.9994


In [11]:
model.save('/kaggle/working/carModel.h5')